In [15]:
import matplotlib
from datetime import datetime
from matplotlib import pyplot as plt
import numpy as np
import pandas
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import LeaveOneOut
from sklearn import linear_model, tree, ensemble

from numpy import mean
from matplotlib import pyplot
import statistics
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import plot_roc_curve
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from matplotlib.pyplot import figure
from keras.optimizers import RMSprop
from sklearn.model_selection import RandomizedSearchCV

# formating
plot_size = (25,10) #set plot size (entire plot)

# get indicies for time data
def get_indices(time_data, start_time, end_time):
    """Return indecies of time_data that are >= start_time and < end_time
    
    time_data  - np array of type float, time_data indices will correlate to
    start_time - float, a timestamp for the start
    end_time   - float, a timestamp for the end
    
    return     - a tuple of indices
    """
    return tuple(np.where(
        np.logical_and(
            np.greater_equal(time_data, start_time), np.less(time_data, end_time)
        ))[0])

def get_user_slice(target, subjects):
    '''return slice for user - the indicies
       Target - array you are slicing 
       Slices are done on target dataset, user must ensure data.shape matches
    '''
    subjects_slice = np.array([target == i for i in subjects]).any(axis=0) #or the list of lists

    return subjects_slice

def kFold(model, train_x, train_y, subjects, IDs):
    k = 5
    num_val_samples = len(subjects) // k
    num_epochs = 500 
    all_scores = [] 
    for i in range(k):
           
        print(f"Processing fold #{i}")
        
        if( i == (k-1)):
            validation_indicies = get_user_slice(target=IDs, subjects=subjects[i * num_val_samples:len(subjects)])
            print("Validation Set: Subjects " + str(i * num_val_samples) + ":" +  str(len(subjects)))

        else:
            validation_indicies = get_user_slice(target=IDs, subjects=subjects[i * num_val_samples: (i + 1) * num_val_samples])
            print("Validation Set: Subjects " + str(i * num_val_samples) + ":" +  str((i + 1) * num_val_samples))
            
        train_indicies = np.invert(validation_indicies)
                                   
        val_data = train_x[validation_indicies]
        val_targets = train_y[validation_indicies]
        
        partial_train_data = train_x[train_indicies]
        partial_train_targets = train_y[train_indicies]
                
        model.fit(partial_train_data, partial_train_targets,                    
                  epochs=num_epochs, batch_size=1, verbose=0)
        loss, val_acc = model.evaluate(val_data, val_targets, verbose=0)     
        all_scores.append(val_acc)
    #print("Fold Accuracies: ", all_scores)
    #print("Average Accuracy: ", np.mean(all_scores))
    
    return np.mean(all_scores)

# participant class that holds datasets for all participants
class Participant:

    def __init__(self, baseline_fileName, coldpressor_fileName,  participantNumber):

        self.cutoff = 99999999
        
        ########################################### Load Baseline Data ############################################
        
        baseline_input_data = pandas.read_csv(baseline_fileName)
        # print(baseline_input_data.head())
        self.unique_baseline_values = baseline_input_data["ibi"].unique()
        baseline_input_data.describe()
        baseline_data = baseline_input_data.values

        # store all values into Participant variables
        self.baseline_participantNumber = participantNumber
        self.baseline_id = baseline_data[:,[0]] # store all rows, id column
        self.baseline_user_id = baseline_data[:,[1]] # store all rows, user_id column
        self.baseline_unix_in_ms = baseline_data[baseline_data[:, 2].argsort()] # store all rows, unix_in_ms column sorted
        self.baseline_ibi = baseline_data[:,[3]] # store all rows, ibi column
        self.baseline_raw_data = baseline_data
        
        # get sorted unix time and ibi to store into plotting variable
        
        baseline_data = baseline_data[:,[2,3]] # Get all rows, time and ibi columns
        baseline_data = baseline_data[baseline_data[:, 0].argsort()]  # Sort by time to ensure time is chronological
        
        # print(baseline_data)
        
        self.baseline_unix_vs_ibi_for_plotting = baseline_data[:,[0,1]] #variable to be used for plotting
       # x axis timing
        baseline_plot_width_seconds = baseline_data[0,0] - baseline_data[0,1]
        baseline_min_time = baseline_data[1,0]
        baseline_max_time = baseline_min_time + baseline_plot_width_seconds

        # define indices and x values
        self.baseline_indices = get_indices(baseline_data[:,0], baseline_min_time, baseline_max_time)
        self.baseline_x_vals = [datetime.utcfromtimestamp(t/1000) for t in baseline_data[self.baseline_indices,0]] #convert to datetime objects

        ########################################### Load Cold Pressor Data ############################################
        
        coldpressor_input_data = pandas.read_csv(coldpressor_fileName)
        self.unique_coldpressor_values = coldpressor_input_data["ibi"].unique()
        coldpressor_input_data.describe()
        coldpressor_data = coldpressor_input_data.values
        # print(input_data)

        # store all values into Participant variables
        self.coldpressor_participantNumber = participantNumber
        self.coldpressor_id = coldpressor_data[:,[0]] # store all rows, id column
        self.coldpressor_user_id = coldpressor_data[:,[1]] # store all rows, user_id column
        self.coldpressor_unix_in_ms = coldpressor_data[coldpressor_data[:, 2].argsort()] # store all rows, unix_in_ms column sorted
        self.coldpressor_ibi = coldpressor_data[:,[3]] # store all rows, ibi column
        self.coldpressor_raw_data = coldpressor_data
        
        # get sorted unix time and ibi to store into plotting variable
        
        coldpressor_data = coldpressor_data[:,[2,3]] # Get all rows, time and ibi columns
        coldpressor_data = coldpressor_data[coldpressor_data[:, 0].argsort()]  # Sort by time to ensure time is chronological
        
        
        self.coldpressor_unix_vs_ibi_for_plotting = coldpressor_data[:,[0,1]] #variable to be used for plotting
        
       # x axis timing
        coldpressor_plot_width_seconds = coldpressor_data[0,0] - coldpressor_data[0,1]
        coldpressor_min_time = coldpressor_data[1,0]
        coldpressor_max_time = coldpressor_min_time + coldpressor_plot_width_seconds

        # define indices and x values
        self.coldpressor_indices = get_indices(coldpressor_data[:,0], coldpressor_min_time, coldpressor_max_time)
        self.coldpressor_x_vals = [datetime.utcfromtimestamp(t/1000) for t in coldpressor_data[self.coldpressor_indices,0]] #convert to datetime objects
        
        self.baseline_data_length = len(self.baseline_unix_vs_ibi_for_plotting[:,1])
        self.coldpressor_data_length = len(self.coldpressor_unix_vs_ibi_for_plotting[:,1])
        
    # getter and setter methods
    
    #plot participant baseline data
    def plot_baseline(self):
    
        # decorate
        fig = plt.figure(self.baseline_participantNumber)
        ax = plt.axes()
        plt.grid()
        plt.title("Participant " + str(self.baseline_participantNumber) + " Baseline Test")
        plt.xlabel('Time (Unix)', fontsize=15)
        plt.ylabel('IBI Measurement (ms)', fontsize=15)
        plt.rcParams['figure.figsize'] = plot_size

        indices = self.baseline_indices

        # plot
        plt.plot(self.baseline_x_vals,self.baseline_unix_vs_ibi_for_plotting[indices,1], 'o', linestyle = '--')
        plt.show()
        plt.close()
        
    #plot participant data
    def plot_coldpressor(self):
    
        # decorate
        fig = plt.figure(self.coldpressor_participantNumber)
        ax = plt.axes()
        plt.grid()
        plt.title("Participant " + str(self.coldpressor_participantNumber) + " Cold Pressor Test")
        plt.xlabel('Time (Unix)', fontsize=15)
        plt.ylabel('IBI Measurement (ms)', fontsize=15)
        plt.rcParams['figure.figsize'] = plot_size

        indices = self.coldpressor_indices

        # plot
        plt.plot(self.coldpressor_x_vals,self.coldpressor_unix_vs_ibi_for_plotting[indices,1], 'o', linestyle = '--', color = 'ORANGE')
        plt.show()
        plt.close()
        
    def plot_both(self):
        
        # decorate
        fig = plt.figure(self.coldpressor_participantNumber)
        ax = plt.axes()
        plt.grid()
        plt.title("Participant " + str(self.coldpressor_participantNumber) + " Baseline And Cold Pressor Test")
        plt.xlabel('Time (Unix)', fontsize=15)
        plt.ylabel('IBI Measurement (ms)', fontsize=15)
        plt.rcParams['figure.figsize'] = plot_size
        
        baseline_indices = self.baseline_indices
        coldpressor_indices = self.coldpressor_indices

        # plot
        line1, = plt.plot(self.baseline_x_vals,self.baseline_unix_vs_ibi_for_plotting[baseline_indices,1], 'o', linestyle = '--', label = 'Baseline Test')
        line2, = plt.plot(self.coldpressor_x_vals,self.coldpressor_unix_vs_ibi_for_plotting[coldpressor_indices,1], 'o', linestyle = '--', color = 'ORANGE', label = 'Cold Pressor Test')
        
        # legend
        plt.legend(handles=[line1, line2], loc='upper left', fontsize = 'x-large')
        
        plt.show()
        plt.close()
        
    def normalizeBaselineData(self):
        
        baseline_normalizedData = self.baseline_unix_vs_ibi_for_plotting #initialize baseline_normalizedData variable
        
        maxBaselineValue = max(self.baseline_unix_vs_ibi_for_plotting[:,1]) #set maxValue to highest value of ibi signals
        minBaselineValue = min(self.baseline_unix_vs_ibi_for_plotting[:,1]) #set minValue to lowest value of ibi signals
        maxColdPressorValue = max(self.coldpressor_unix_vs_ibi_for_plotting[:,1]) #set maxValue to highest value of ibi signals
        minColdPressorValue = min(self.coldpressor_unix_vs_ibi_for_plotting[:,1]) #set minValue to lowest value of ibi signals
        maxValue = max(maxBaselineValue, maxColdPressorValue)
        minValue = min(minBaselineValue, minColdPressorValue)
        
        
        numRows = len(self.baseline_unix_vs_ibi_for_plotting) #get number of rows
        
        for i in range(numRows):
            baseline_normalizedData[i,1] = (self.baseline_unix_vs_ibi_for_plotting[i,1] - minValue)/(maxValue-minValue)
        
        self.baseline_unix_vs_ibi_for_plotting = baseline_normalizedData
        return baseline_normalizedData
        
    def normalizeColdpressorData(self):
        
        coldpressor_normalizedData = self.coldpressor_unix_vs_ibi_for_plotting #initialize coldpressor_normalizedData variable
        
        maxBaselineValue = max(self.baseline_unix_vs_ibi_for_plotting[:,1]) #set maxValue to highest value of ibi signals
        minBaselineValue = min(self.baseline_unix_vs_ibi_for_plotting[:,1]) #set minValue to lowest value of ibi signals
        maxColdPressorValue = max(self.coldpressor_unix_vs_ibi_for_plotting[:,1]) #set maxValue to highest value of ibi signals
        minColdPressorValue = min(self.coldpressor_unix_vs_ibi_for_plotting[:,1]) #set minValue to lowest value of ibi signals
        maxValue = max(maxBaselineValue, maxColdPressorValue)
        minValue = min(minBaselineValue, minColdPressorValue)
        
        numRows = len(self.coldpressor_unix_vs_ibi_for_plotting) #get number of rows
        
        for i in range(numRows):
            coldpressor_normalizedData[i,1] = (self.coldpressor_unix_vs_ibi_for_plotting[i,1] - minValue)/(maxValue-minValue)
        
        self.coldpressor_unix_vs_ibi_for_plotting = coldpressor_normalizedData
        return coldpressor_normalizedData
    
#     def createTrainTestColdPressorDataset(self):
        
#         trainColdPressorDataset, testColdPressorDataset = train_test_split(self.coldpressor_unix_vs_ibi_for_plotting[:,:], shuffle=False, train_size = .75)
        
#         self.trainColdPressorDataset = trainColdPressorDataset
#         self.testColdPressorDataset = testColdPressorDataset
    
#     def createTrainTestBaselineDataset(self):
        
#         trainBaselineDataset, testBaselineDataset = train_test_split(self.baseline_unix_vs_ibi_for_plotting[:,:], shuffle=False, train_size = .75)
        
#         self.trainBaselineDataset = trainBaselineDataset
#         self.testBaselineDataset = testBaselineDataset
    
#     def splitDataSampleLevel(self):
        
#         if(self.baseline_data_length > self.coldpressor_data_length):
#             length = self.coldpressor_data_length
#             self.baseline_unix_vs_ibi_for_plotting = np.delete(self.baseline_unix_vs_ibi_for_plotting, slice(length,self.baseline_data_length), axis=0)
#             self.baseline_data_length = len(self.baseline_unix_vs_ibi_for_plotting[:,1])
#             return self.baseline_unix_vs_ibi_for_plotting
#         else:
#             length = self.baseline_data_length
#             self.coldpressor_unix_vs_ibi_for_plotting = np.delete(self.coldpressor_unix_vs_ibi_for_plotting, slice(length,self.coldpressor_data_length), axis=0)
#             self.coldpressor_data_length = len(self.coldpressor_unix_vs_ibi_for_plotting[:,1])
#             return self.coldpressor_unix_vs_ibi_for_plotting
        
    def splitDataKeepLeft(self, cutoff):
    
            self.baseline_unix_vs_ibi_for_plotting = np.delete(self.baseline_unix_vs_ibi_for_plotting, slice(cutoff,self.baseline_data_length), axis=0)
            self.coldpressor_unix_vs_ibi_for_plotting = np.delete(self.coldpressor_unix_vs_ibi_for_plotting, slice(cutoff,self.coldpressor_data_length), axis=0)

            self.baseline_data_length = len(self.baseline_unix_vs_ibi_for_plotting[:,1])
            self.coldpressor_data_length = len(self.coldpressor_unix_vs_ibi_for_plotting[:,1])
            
        
    def splitDataKeepRight(self, cutoff):
    
            self.baseline_unix_vs_ibi_for_plotting = np.delete(self.baseline_unix_vs_ibi_for_plotting, slice(0,self.baseline_data_length-cutoff), axis=0)
            self.coldpressor_unix_vs_ibi_for_plotting = np.delete(self.coldpressor_unix_vs_ibi_for_plotting, slice(0,self.coldpressor_data_length-cutoff), axis=0)

            self.baseline_data_length = len(self.baseline_unix_vs_ibi_for_plotting[:,1])
            self.coldpressor_data_length = len(self.coldpressor_unix_vs_ibi_for_plotting[:,1])

        
    def getfeatures(self):            
        
        # Using Skicit-learn to split data into training and testing sets
        # Split the data into training and testing sets
        train_features, test_features, train_labels, test_labels = train_test_split(self.features, self.labels, test_size = 0.25, random_state = 42)
        
        # Instantiate model with 1000 decision trees
        rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
        
        # Train the model on training data
        rf.fit(train_features, train_labels);
        
    # id getter method
    def get_baseline_unix_vs_ibi_for_plotting(self):
        return self.baseline_unix_vs_ibi_for_plotting

    # user_id getter method
    def getUser_ID(self):
        return self.user_id

    # unix_in_ms getter method
    def getUnix_in_ms(self):
        return self.unix_in_ms

    # ibi getter method
    def getIbi(self):
        return self.ibi
    
    # ibi getter method
    def getData(self):
        return self.data

    # id setter method
    def setID(self, id):
        self.id = id

    # user_id setter method
    def setUser_ID(self, user_id):
        self.user_id = user_id

    # unix_in_ms setter method
    def setUnix_in_ms(self, unix_in_ms):
        self.unix_in_ms = unix_in_ms

    # ibi setter method
    def setIbi(self, ibi):
        self.ibi = ibi
        
    def getBaselineSize(self):
        return self.baseline_data_length
    
    def getColdpressorSize(self):
        return self.coldpressor_data_length
    
p = 0
rf_accuracies = []
rf_kfold_accuracies = []
dense_accuracies = []
dense_kfold_accuracies = []
regression_accuracies = []

while(p<10):
    #main
    
    print("Running Trial #" + str(p))

    #init participants
    participants = [Participant for i in range(61)]

    #store all participant data
    n = 1
    while (n<61):
        if(n != 2 and (n < 17 or n > 31) and n != 5): #No participants 2,17-31. Participant 5 has missing values
            participants[n] = Participant("C:\\Users\\C23Price.Johnson\\Desktop\\Capstone\\Capstone Project\\Pain-Identification\\src\\raw_data\\raw_data_garmin_unique_values\\participant_" + str(n) + "_BL.csv",
                                          "C:\\Users\\C23Price.Johnson\\Desktop\\Capstone\\Capstone Project\\Pain-Identification\\src\\raw_data\\raw_data_garmin_unique_values\\participant_" + str(n) + "_CPT.csv",n)
        n = n + 1

    # Normalize Data
    n = 1
    while (n<61):
        if(n != 2 and (n < 17 or n > 31) and n != 5): #No participants 2,17-31. Participant 5 has missing values
            participants[n].normalizeBaselineData()
            participants[n].normalizeColdpressorData()
        n = n + 1

    #initiate variables
    baselineTotal = 0
    coldPressorTotal = 0
    numParticipants = 0
    smallestBaselineSize = 99999999
    smallestColdpressorSize = 99999999
    cutoff = 200
    numberOfUsableParticipants = 0

    #get average number of baseline and coldpressor datapoints
    n = 1
    while (n<61):
        if(n != 2 and (n < 17 or n > 31) and n != 5): #No participants 2,17-31. Participant 5 has missing values
            baselineTotal = baselineTotal + participants[n].getBaselineSize()
            coldPressorTotal = coldPressorTotal + participants[n].getColdpressorSize()
            numParticipants = numParticipants + 1
            smallestBaselineSize = min(smallestBaselineSize, participants[n].baseline_data_length)
            smallestColdpressorSize = min(smallestColdpressorSize, participants[n].coldpressor_data_length)
        n = n + 1

    averageNumberOfBaselineDatapoints = round((baselineTotal / numParticipants), 0)
    averageNumberOfColdpressorDatapoints = round((coldPressorTotal / numParticipants), 0)

    IDs = []
    lengths = []
    n=1
    while (n<61):
        if(n != 2 and (n < 17 or n > 31) and n != 5): #No participants 2,17-31. Participant 5 has missing values
            IDs.append(n)
            lengths.append(participants[n].coldpressor_data_length)
        n = n + 1

    # Get all usable participant data that where usable participant's dataset size if greater than the cutoff
    usableParticipants = [Participant for i in range(1)]
    n = 1
    while (n<61 and participants[n] != None):
        if(n != 2 and (n < 17 or n > 31) and n != 5): #No participants 2,17-31. Participant 5 has missing values
            if(((participants[n].getBaselineSize() >= cutoff) and (participants[n].getColdpressorSize() >= cutoff)) or (not participants[n])):
                usableParticipants.append(participants[n])
                numberOfUsableParticipants = numberOfUsableParticipants + 1
        n = n + 1

    #split data into same size
    n = 1
    while (n<=numberOfUsableParticipants):
        usableParticipants[n].splitDataKeepRight(cutoff)
        n = n + 1

    # create samples and labels array based on usable participants
    samples = [usableParticipants[1].baseline_unix_vs_ibi_for_plotting for i in range(1)]
    samples[0] = usableParticipants[1].baseline_unix_vs_ibi_for_plotting[:,1]
    labels = [0 for x in range(0, numberOfUsableParticipants)]
    ID = []
    ID.append(1)
    n = 2
    while (n<=numberOfUsableParticipants):
        samples.append(usableParticipants[n].baseline_unix_vs_ibi_for_plotting[:,1])
        ID.append(n)
        n = n + 1 
    n = 1
    while (n<=numberOfUsableParticipants):
        samples.append(usableParticipants[n].coldpressor_unix_vs_ibi_for_plotting[:,1])
        labels.append(1)
        ID.append(n)
        n = n + 1

    # Saving feature names for later use
    feature_names = ["IBI", "Pain"]

    #parallel arrays, same indicies
    samples = np.array(samples).astype(np.float32)
    labels = np.array(labels)
    ID = np.array(ID)

    subjects = ID[0:numberOfUsableParticipants] # subjects to shuffle on

    cut = .2

    np.random.shuffle(subjects)
    split_count = int(cut * numberOfUsableParticipants) #porportion of train and test splits

    #indicies from 0 to split_count
    validation_indicies = get_user_slice(target=ID, subjects=subjects[0:split_count]) 

    #indicies from split_count to 2xsplit_count
    test_indicies = get_user_slice(target=ID, subjects=subjects[split_count:2 * split_count]) 
    train_indicies = np.invert(np.array([test_indicies,validation_indicies]).any(axis=0))

    train_x = samples[train_indicies]
    train_y = labels[train_indicies]
    val_x   = samples[validation_indicies]
    val_y   = labels[validation_indicies]
    test_x  = samples[test_indicies]
    test_y  = labels[test_indicies]

    kfold_subjects = subjects
    k_fold_IDs_indicies = np.invert(np.array(test_indicies))
    k_fold_IDs = ID[k_fold_IDs_indicies]
    temp_array = subjects[split_count:2 * split_count]

    i = 0
    j = 0
    while i < (kfold_subjects.size):
        j = 0
        while j < (temp_array.size):
            if(kfold_subjects[i] == temp_array[j]):
                kfold_subjects = np.delete(kfold_subjects,i)
            j = j + 1
        i = i + 1

    # Instantiate model with 1000 decision trees, use classifier for binary classification 
    rf = RandomForestClassifier(n_estimators = 1000, random_state = 42)

    # Train the model on training data
    rf.fit(train_x, train_y);

    # Use the forest's predict method on the test data
    predictions = rf.predict(test_x)

    # Find which values were correctly predicted
    correctPredictions =  [0 for x in range(0, len(test_y))]
    n = 0
    numCorrect = 0
    while (n<len(test_y)):
        correctPredictions[n] = int (not (test_y[n] ^ predictions[n]))
        if(correctPredictions[n] == 1):
            numCorrect = numCorrect + 1
        n = n + 1

    #K fold
    kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    score = cross_val_score(ensemble.RandomForestClassifier(random_state= 42), train_x, train_y, cv= kf, scoring="accuracy")

    #print("Accuracy: " + str( round( ((numCorrect/len(test_y)) * 100), 2)) + "%")
    #print(f'K Fold Average score: {"{:.2f}".format(score.mean()*100)}' + '%')
    
    rf_accuracies.append(round( ((numCorrect/len(test_y)) * 100), 2))
    rf_kfold_accuracies.append(round(score.mean()*100, 2))
    
    k_train_x = np.vstack((train_x, val_x))
    k_train_y = np.hstack((train_y, val_y))
    
    # Practice with Keras Model
    # Activation and Loss Function
    model = Sequential()
    #model.add(Dense(1, input_shape=(cutoff,), activation='relu', kernel_initializer = 'he_normal', kernel_regularizer = 'l1'))
    #model.add(Dense(128, activation='relu', kernel_initializer = 'he_normal', kernel_regularizer = 'l2'))
    # model.add(Dense(64, activation='relu', kernel_initializer = 'he_normal', kernel_regularizer = 'l2'))
    #model.add(Dense(32, activation='relu', kernel_initializer = 'he_normal', kernel_regularizer = 'l2'))
    #model.add(Dense(16, activation='relu', kernel_initializer = 'he_normal', kernel_regularizer = 'l2'))
    model.add(Dropout(rate=0.2))
    model.add(Dense(8, activation='relu', kernel_initializer = 'he_normal', kernel_regularizer = 'l1'))
    #model.add(Dense(4, activation='relu', kernel_initializer = 'he_normal', kernel_regularizer = 'l1'))
    model.add(Dropout(rate=0.4))
    #model.add(Dense(2, activation='relu', kernel_initializer = 'he_normal', kernel_regularizer = 'l2'))
    model.add(Dense(1, activation='sigmoid', kernel_initializer = 'he_normal'))
    model.compile(loss='binary_crossentropy', optimizer=RMSprop(learning_rate = .00025), metrics=['acc'])
    #model.summary()

    # fit the keras model on the dataset
    history = model.fit(train_x, train_y, validation_data=(val_x, val_y), epochs=800, batch_size=1, verbose=0)

    # Print accuracy of the model, anything greater than 50% would be great
    _, accuracy = model.evaluate(test_x, test_y)
    print('Accuracy: %.2f' % (accuracy*100))

    #K fold

    k_train_x = np.vstack((train_x, val_x))
    k_train_y = np.hstack((train_y, val_y))

    #kFold(model, k_train_x, k_train_y, kfold_subjects, k_fold_IDs)
    dense_accuracies.append(round( (accuracy*100), 2))
    dense_kfold_accuracies.append(round(kFold(model, k_train_x, k_train_y, kfold_subjects, k_fold_IDs)*100, 2))
    
    # instantiate the model (using the default parameters)
    model = LogisticRegression(random_state=0)

    # fit the model with data
    model.fit(train_x, train_y)

    pred_y = model.predict(test_x)

    #print("Before Grid Accuracy: " + str(model.score(test_x , test_y) * 100) + "%")

    # messing around with grid to see if it improves accuracy

    # instantiate parameters
    params = {
    "max_iter": [20, 50, 100, 200],
    "solver": ["newton-cg", "lbfgs", "liblinear", "sag", "saga"],
    "class_weight": ["balanced"]
    }

    # grid fit
    logModel_grid = GridSearchCV(estimator=LogisticRegression(random_state=0),scoring ="accuracy", param_grid=params, verbose=1, cv=10, n_jobs=2)
    logModel_grid.fit(train_x, train_y)

    # grid predict
    pred_y_grid = logModel_grid.predict(test_x)

    #print("After Grid Accuracy: " + str(accuracy_score(test_y, pred_y_grid) * 100) + "%")

    regression_accuracies.append(round(accuracy_score(test_y, pred_y_grid) * 100))

    p = p + 1
print("Random Forrests Accuracies: ")
print(rf_accuracies)
print("Random Forrests KFold Accuracies: ")
print(rf_kfold_accuracies)
print("Dense Accuracies: ")
print(dense_accuracies)
print("Dense KFold Accuracies: ")
print(dense_kfold_accuracies)
print("Regression Accuracies: ")
print(regression_accuracies)

Running Trial #0
1/1 [==============================] - 0s 136ms/step - loss: 0.3650 - acc: 0.9000
Accuracy: 90.00
Processing fold #0
Validation Set: Subjects 0:4
Processing fold #1
Validation Set: Subjects 4:8
Processing fold #2
Validation Set: Subjects 8:12
Processing fold #3
Validation Set: Subjects 12:16
Processing fold #4
Validation Set: Subjects 16:21
Fitting 10 folds for each of 20 candidates, totalling 200 fits
Running Trial #1
1/1 [==============================] - 0s 135ms/step - loss: 0.5047 - acc: 0.8000
Accuracy: 80.00
Processing fold #0
Validation Set: Subjects 0:4
Processing fold #1
Validation Set: Subjects 4:8
Processing fold #2
Validation Set: Subjects 8:12
Processing fold #3
Validation Set: Subjects 12:16
Processing fold #4
Validation Set: Subjects 16:21
Fitting 10 folds for each of 20 candidates, totalling 200 fits
Running Trial #2
1/1 [==============================] - 0s 169ms/step - loss: 0.3617 - acc: 0.9000
Accuracy: 90.00
Processing fold #0
Validation Set: Subj

C:\Users\C23PRI~1.JOH\AppData\Local\R-MINI~1\envs\cap2\lib\site-packages\scipy\optimize\linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\C23PRI~1.JOH\AppData\Local\R-MINI~1\envs\cap2\lib\site-packages\sklearn\utils\optimize.py:203: UserWarning: Line Search failed
  warnings.warn("Line Search failed")


Running Trial #7
1/1 [==============================] - 0s 119ms/step - loss: 0.3159 - acc: 0.9000
Accuracy: 90.00
Processing fold #0
Validation Set: Subjects 0:4


KeyboardInterrupt: 